# Qwen3 Agent with OpenVINO GenAI & Smolagents

## Accelerated AI Agent Deployment with Speculative Decoding

This notebook demonstrates how to implement an intelligent agent using **Qwen3-8B** with **HuggingFace's smolagents** framework and **Intel's OpenVINO GenAI** library. We'll showcase how speculative decoding can significantly accelerate agent inference, achieving up to **1.6x faster performance** compared to standard auto-regressive generation on Intel AI PCs.

### What We'll Build

- **Qwen3 Agent**: A conversational AI agent powered by the Qwen3-8B language model
- **Tool Integration**: Using smolagents framework for seamless tool calling capabilities
- **Performance Optimization**: Leveraging OpenVINO GenAI's speculative decoding for faster inference
- **Interactive Demo**: A Gradio-based web interface for real-time agent interaction

### Key Technologies

- **🤖 Qwen3-8B**: Advanced language model from Alibaba with strong reasoning capabilities
- **🔧 HuggingFace smolagents**: Lightweight framework for building AI agents with tool-calling abilities
- **⚡ Intel OpenVINO GenAI**: High-performance inference library with speculative decoding optimization
- **🖥️ Gradio**: User-friendly web interface for interactive demonstrations
- **💻 Intel AIPC**: Optimized deployment on Intel AI-accelerated hardware

### Performance Benefits

By utilizing OpenVINO GenAI's speculative decoding, we achieve:
- **1.6x faster inference** compared to standard auto-regressive generation
- **Reduced latency** for real-time agent interactions
- **Optimized resource utilization** on Intel AI PC hardware

Let's get started by setting up our environment and implementing the agent!

## 🚀 Environment Setup

Before we begin, let's set up a clean Python environment and install the required dependencies.

### Step 1: Create a New Python Environment

We recommend creating a new virtual environment to avoid conflicts with existing packages:

```bash
# Create a new conda environment (recommended)
conda create -n qwen3-agent python=3.11 -y
conda activate qwen3-agent

# OR create a virtual environment with venv
python -m venv qwen3-agent
# On Windows:
qwen3-agent\Scripts\activate
# On Linux:
source qwen3-agent/bin/activate
```

### Step 2: Install Dependencies

Install the required packages from the requirements files:

```bash
# Install main dependencies
pip install -r requirements.txt

# Install additional packages for this notebook
pip install smolagents[openai,mcp,gradio] ddgs markdownify requests ipython ipykernel ipywidgets
```

Once you have your environment set up and dependencies installed, you're ready to proceed with the implementation!

## 📥 Download the Qwen3-8B Model

Next, we need to download the pre-optimized Qwen3-8B model in OpenVINO format. This model has been quantized to INT4 for optimal performance on Intel hardware.

The model will be downloaded from HuggingFace Hub and stored locally for use with OpenVINO GenAI.

In [ ]:
import huggingface_hub as hf_hub

model_id = "OpenVINO/Qwen3-8B-int4-ov"
model_path = "./qwen3-8b-int4-ov"

hf_hub.snapshot_download(model_id, local_dir=model_path)

## 🚀 Download the Draft Model for Speculative Decoding

For speculative decoding acceleration, we also need to download a smaller draft model (Qwen3-0.6B). This smaller model generates initial predictions that are then verified by the main model, significantly speeding up inference.

The draft model works in tandem with the main model to achieve up to 1.6x faster generation speeds.

In [ ]:
# Download the draft model for speculative decoding
draft_model_id = "OpenVINO/Qwen3-0.6B-int8-ov"
draft_model_path = "./qwen3-0.6b-int8-ov"

hf_hub.snapshot_download(draft_model_id, local_dir=draft_model_path)

## 🖥️ Start the OpenVINO GenAI Server

Now we'll start our OpenVINO GenAI server with speculative decoding enabled. 

The server will expose an OpenAI-compatible API endpoint that we can use with smolagents.

In [ ]:
import subprocess
import time

# Start the server with speculative decoding
server_process = subprocess.Popen([
    "python", "server.py", 
    "--model_path", model_path,
    "--draft_path", draft_model_path
])

# Wait a moment for the server to start
time.sleep(20)

print(f"Server started with PID: {server_process.pid}")

## 🤖 Initialize the Smolagents Demo

Now that our server is running, we can create a Qwen3 agent using smolagents. The agent will communicate with our OpenVINO GenAI server through OpenAI-compatible API calls.

We'll set up:
1. A model wrapper that sends requests to our local server
2. A tool-calling agent with access to basic tools from smolagents

In [ ]:
from smolagents import OpenAIServerModel, ToolCallingAgent

# Configuration
model_id = "Qwen3-8B"
host = "localhost"
port = 8000
enable_thinking = True  # Enable Qwen3's thinking capability

In [ ]:
# Configure thinking and generation parameters according to Qwen3 recommendations
extra_body = {"chat_template_kwargs": {"enable_thinking": enable_thinking}}

# Use sampling parameters (recommended for thinking mode)
if enable_thinking:
    generation_params = {
        'temperature': 0.6,
        'top_p': 0.95,
    }
else:
    generation_params = {
        'temperature': 0.7,
        'top_p': 0.8,
    }

# Initialize the model wrapper
api_base = f'http://{host}:{port}/v1'
model = OpenAIServerModel(
    model_id,
    api_base=api_base,
    api_key='None',
    max_tokens=4096,
    extra_body=extra_body,
    **generation_params
)

In [ ]:
# Initialize the tool-calling agent
agent = ToolCallingAgent(
    tools=[],  # Start with no custom tools
    model=model, 
    add_base_tools=True,  # Include basic smolagents toolbox
    stream_outputs=True,  # Enable streaming for real-time responses
    planning_interval=None,
)

## 🎯 Test the Agent

Let's test our Qwen3 agent by asking it about OpenVINO GenAI. The agent will use its thinking capabilities and available tools to provide a comprehensive response.

In [ ]:
agent.run("What is OpenVINO GenAI? What are the latest features?", reset=True)

## 🚀 Launch Interactive Gradio Demo

Now let's create an interactive web interface using Gradio. This will provide a user-friendly chat interface where you can interact with the Qwen3 agent in real-time, showcasing the accelerated performance from OpenVINO GenAI's speculative decoding.

In [ ]:
from smolagents import GradioUI

# Create and launch the Gradio interface
demo = GradioUI(agent)
demo.launch(share=False)

## 🎉 Summary

Congratulations! You've successfully implemented and deployed a high-performance Qwen3 agent with accelerated inference. Here's what we accomplished:

### What We Built:
- **OpenVINO GenAI Server**: Deployed Qwen3-8B with speculative decoding using a 0.6B draft model
- **Smolagents Integration**: Created a tool-calling agent with access to base tools
- **Interactive Interface**: Launched a Gradio web UI for real-time interaction

### Key Performance Benefits:
- **1.6x faster inference** compared to standard auto-regressive generation
- **Optimized for Intel hardware** using OpenVINO's acceleration
- **Real-time tool calling** with thinking capabilities enabled

### Technical Stack:
- **Intel OpenVINO GenAI** for optimized inference
- **HuggingFace Smolagents** for agent framework
- **Qwen3 models** (8B target + 0.6B draft) for speculative decoding
- **Gradio** for interactive web interface

### Expanding Agent Capabilities:
With smolagents, you can easily extend your agent by:
- **Custom Tools**: Add domain-specific tools for specialized tasks
- **MCP Integration**: Connect to Model Context Protocol servers for external capabilities
- **Sub-Agents**: Create hierarchical agent systems with specialized sub-agents

You now have a working demonstration of accelerated AI agent inference on Intel AI PCs with tool-calling capabilities!